# Spark activity 4

In [3]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import xml.etree.ElementTree as ET

# Initialize spark driver
findspark.init()
spark = SparkSession.builder.master("local").appName("FSTExample").getOrCreate()

# TO read the whole XML file as a single string 
file_rdd = spark.read.text('persons.xml', wholetext=True).rdd

# Function to parse the file and extract data
ELEMENTS_TO_EXTRACT = ['firstname', 'middlename', 'lastname', 'dob_year', 'dob_month', 'salary', 'gender']
def parse_xml(rdd):
    """
    This function is used to read an xml string from rdd, parse and extract the elements,
    then return a list of lists.
    """
    results = []
    root = ET.fromstring(rdd[0])

    for p in root.findall('person'):
        rec = []
        rec.append(p.attrib['id'])

        for e in ELEMENTS_TO_EXTRACT:
            if p.find(e) is None:
                rec.append(None)
                continue
            value = p.find(e).text
            if e == 'salary':
                value = float(value)
            rec.append(value)
        results.append(rec)
    return results
  
# To view the data frame
records_rdd = file_rdd.flatMap(parse_xml)
persons = records_rdd.toDF(("id", "firstName", "middleName", "lastName", "dob_year", "dob_month", "salary", "gender"))
persons.show()

+---+---------+----------+--------+--------+---------+---------+------+
| id|firstName|middleName|lastName|dob_year|dob_month|   salary|gender|
+---+---------+----------+--------+--------+---------+---------+------+
|  1|    James|   Madison|   Smith|    1980|        1|  10000.0|     M|
|  2|     Rose|      NULL|   Tyler|    1990|        6|  10000.0|     M|
|  3|     Lily|      NULL|   Pichu|    1989|       11|1000000.0|     F|
+---+---------+----------+--------+--------+---------+---------+------+

